<a href="https://colab.research.google.com/github/bird0401/Instance_level_recognition/blob/main/scraping_from_wikimedia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install fake_useragent

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=da39453df4bad0977ac48d940913e951d5dd45e157a3a4be02c8516c33a31c7f
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent


In [17]:
import requests
from bs4 import BeautifulSoup
import re
import os
import shutil
from fake_useragent import UserAgent
import pathlib
import time

In [3]:
def download_images(url, file_path):
  ua = UserAgent()
  header = {'user-agent':ua.chrome}
  r = requests.get(url, stream=True, headers=header)

  if r.status_code == 200:
    with open(file_path, "wb") as f:
      f.write(r.content)
  else: print(f'error {r.status_code}')

In [14]:
# collect instance urls
first_loop=True
move_to_next_page=True
instance_urls=[]

while move_to_next_page:
  # requests to wikimedia regarding category which I want to exploit images
  wikimedia_url = 'https://commons.wikimedia.org'
  category='Dog_breeds_by_name'
  category_url=f'/wiki/Category:{category}'
  
  if first_loop: 
    res = requests.get(wikimedia_url+category_url)
    time.sleep(1)
    first_loop=False
  else: res = requests.get(wikimedia_url+next_page_url)

  soup = BeautifulSoup(res.text, "html.parser")
  try:  next_page_url=soup.find_all(text="next page")[0].parent.attrs['href']
  except: move_to_next_page=False

  elems=soup.find_all(class_="CategoryTreeItem")

  for elem in elems:
    instance_url=elem.find('a').attrs['href']
    instance_urls.append(instance_url)

In [18]:
# len(instance_urls)

579

In [16]:
# collect image urls of each instance
for instance_url in instance_urls:
  res = requests.get(wikimedia_url+instance_url)
  time.sleep(1)
  soup = BeautifulSoup(res.text, "html.parser")

  extiw_class=soup.select('a[href^="https://www.wikidata.org/wiki/Q"]')
  try:  wikidata_id=pathlib.Path(extiw_class[0].attrs['href']).stem
  except:  continue
  path = "/content/imgs/" + wikidata_id
  os.makedirs(path,exist_ok=True)

  img_urls=[]
  move_to_next_page=True

  while move_to_next_page:
    res = requests.get(wikimedia_url+instance_url)
    time.sleep(1)
    soup = BeautifulSoup(res.text, "html.parser")

    try:  instance_url=soup.find_all(text="next page")[0].parent.attrs['href']
    except: move_to_next_page=False
    
    image_classes=soup.find_all(class_="image")
    for image_class in image_classes:
      img_url=image_class.img.attrs['src']
      img_urls.append(img_url)
      

  for index, url in enumerate(img_urls):
      filename = 'image_' + str(index) + '.jpg'
      image_path = os.path.join(path, filename)
      print(url)
      print(image_path)
      download_images(url=url, file_path=image_path)
  print()

https://upload.wikimedia.org/wikipedia/commons/thumb/1/17/Affenpinscher.jpg/250px-Affenpinscher.jpg
/content/imgs/Q7254/image_0.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/8/85/2013_Westminster_Kennel_Club_Dog_Show-_Affenpinscher_Joey_aka_GCH_Banana_Joe_V_Tani_Kazari_%288471258772%29.jpg/80px-2013_Westminster_Kennel_Club_Dog_Show-_Affenpinscher_Joey_aka_GCH_Banana_Joe_V_Tani_Kazari_%288471258772%29.jpg
/content/imgs/Q7254/image_1.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/d/d8/Affenpinscher_%28Monkey_Terrier%29_from_1915.JPG/120px-Affenpinscher_%28Monkey_Terrier%29_from_1915.JPG
/content/imgs/Q7254/image_2.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/d/d8/Affenpinscher_circa_1915.jpg/120px-Affenpinscher_circa_1915.jpg
/content/imgs/Q7254/image_3.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/2/25/Affenpinscher_dog.jpg/120px-Affenpinscher_dog.jpg
/content/imgs/Q7254/image_4.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/9/94/Aff

KeyboardInterrupt: ignored